In [43]:
from sympy.logic.boolalg import to_cnf, And, Or, Not, Implies, Equivalent
from sympy.abc import A, B, D
from typing import List
from sympy import *
from copy import deepcopy

In [39]:
class Belief_Base:

    def __init__(self):
        self.beliefBase = [] # Storing beliefs put into the belief base as list of lists
        self.CNF = [] # Storing CNF form of beliefs put into the belief base as list of lists
        self.beliefBaseVariableLimit = 8 # If there are more than 8 variables the flag must be set to True to simplify (default is False)
        
    #def form_to_cnf(self, formula: str) -> None:
    def form_to_cnf(self, formula):
        formula = formula
        cnf = to_cnf(formula)
        #print(self.cnf)
        return cnf 
    
    def query_belief(self, belief): # Checks if a belief is present in the belief base 
        if belief in self.beliefs:
            return True
        else:
            return False
    
    def add(self, belief, score):
        belief = belief.replace(' ', '')  # Remove whitespace
        #belief = simplify(belief)
        self.beliefBase.append([belief, score])
        
        CNF=self.form_to_cnf(belief)
        self.CNF.append(CNF)

        
        #new_CNF = []
        #for i in range(len(CNF)):
           # if not is_tautology(CNF[i]):
                # Check that the CNF clause is not a tautology, and reduce it before adding
         #   new_CNF.append(CNF[i])
        #self.CNF.append(new_CNF_)
    
    
    def contract(self, belief):
        # Remove belief from belief base
        for i in range(len(self.beliefBase)):
            if self.beliefBase[i][0] == belief:
                self.beliefBase.pop(i)
                self.CNF.pop(i)
                break
        
        # Remove any other beliefs that are logically entailed by the belief
        new_belief_base = []
        new_CNF = []
        for b, c in self.beliefBase:
            if not implies(belief, b):
                new_belief_base.append([b, c])
                new_CNF.append(self.form_to_cnf(b))
        self.beliefBase = new_belief_base
        self.CNF = new_CNF


    def check_resolvable(self, c1,c2):
        c1_list = list(c1.args)
        if(len(c1_list)<2):
            c1_list = [c1]

        c2_list = list(c2.args)
        if(len(c2_list)<2):
            c2_list = [c2]

        # print("checking resolvability function: ",c1_list, c2_list)
        for c1_elem in c1_list:
            for c2_elem in c2_list:
                if c1_elem == Not(c2_elem):
                    # print("True")
                    return True
                    
                    
        # print("False")
        return False


    def presolve(self, c1, c2):
        # print("Entered presolve")
        flag = 0
        c1_list = list(c1.args)
        if(len(c1_list)<2):
            c1_list = [c1]

        c2_list = list(c2.args)
        if(len(c2_list)<2):
            c2_list = [c2]
        # print(c1_list, c2_list)
        
        while(self.check_resolvable(c1, c2)):
            for c1_elem in c1_list:
                for c2_elem in c2_list:
                    if c1_elem == Not(c2_elem):
                        c1_list.remove(c1_elem)
                        c2_list.remove(c2_elem)
                        # print(c1_list, c2_list)
                        if(len(c1_list)>0 and len(c1_list)>0):
                            c1 = Or(*c1_list)
                            c2 = Or(*c2_list)
                        else:
                            if len(c1_list) == 0:
                                return(Or(*self.unique(c2_list)))
                            else:
                                return(Or(*self.unique(c1_list)))
                
        combined = Or(*self.unique(c1_list + c2_list))

        return combined
    


    def check_subset(self, new, clause_list):

        for n in new:
            if n not in clause_list:
                return False
        
        return True



    # Alpha should be 
    def entailement(self, alpha):
        KB_alpha = self.conjugation_KB_alpha(alpha) # gives KB ∧ ¬α
        # print("KB_alpha: ", KB_alpha)
        KB_alpha_CNF = self.form_to_cnf(KB_alpha)
        # print("KB_alpha_CNF: ", KB_alpha_CNF)
        clauses = KB_alpha_CNF.args # gives the clauses in KB ∧ ¬α
        # print("clauses: ", clauses)
        clauses_list = list(clauses)
        # print("clauses_list: ", clauses_list)
        
        
        while True:
            new = []
            for i in range(0,len(clauses_list)):
                for j in range(i+1, len(clauses_list)):
                    # print(f"Checking {clauses_list[i]} and {clauses_list[j]}")
                    if(self.check_resolvable(clauses_list[i], clauses_list[j])):
                        # print(f"Resolving {clauses_list[i]} and {clauses_list[j]}")
                        resolvents = self.presolve(clauses_list[i], clauses_list[j])
                        # print("Result of resolution: ", resolvents)
                        if resolvents == False:
                            return True
                        new.append(resolvents)
                        # print("New after appending: ", new)

            # print(f"Checking if {new} is a subset of {clauses_list}")
            if(self.check_subset(new, clauses_list)):
                # print(f"{new} is a subset of {clauses_list}")
                return False
            # print(f"{new} is not a subset of {clauses_list}")
            clauses_list = self.unique(clauses_list + new)
            # print("clauses+new = ", clauses_list)
            




    def conjugation_KB_alpha(self, alpha):
        KB = And(*self.CNF)
        return And(KB, Not(alpha))
    
    def unique(self, a):
        un = list(set(a))
        return un
    

    def consequence(self):
        
        KB_list = deepcopy(self.beliefBase)
        while True:
            
            new = []
            for i in range(0,len(KB_list)):
                for j in range(0, len(KB_list)):
                    if i == j:
                        continue

                    else:
                        # Hypothetical syllogism
                        if isinstance(KB_list[i], Implies) and isinstance(KB_list[j], Implies):
                            print(KB_list[i].args, KB_list[j].args)

In [42]:
form= "~(A | B) | D"
form2= "r >> (p | s)"
alpha = "~(A | B | C) | D"
# f1 = "A | B"
# f2 = "B | A"

bb = Belief_Base()
# bb.add(form2,0.1)
# bb.add("p",0.1)
# bb.add(f1, 0.1)
bb.add("A",0.1)
bb.add("A>>B", 0.1)


print(bb.beliefBase)
p_not = "p>>q"
# print(bb.entailement(f2))
print(bb.entailement("~B"))

[['A', 0.1], ['A>>B', 0.1]]
False


In [ ]:
form= "~(A | B) | D"
form2= "r >> (p | s)"
alpha = "~(A | B | C) | D"


bb = Belief_Base()
bb.add(form2,0.1)
bb.add("p",0.1)
conj_all =  bb.conjugation_KB_alpha(alpha)
cnf_conj_all = bb.form_to_cnf(conj_all)
print(conj_all)
print(cnf_conj_all)

for a in cnf_conj_all.args:
    print(a)

# clauses = cnf_conj_all.split('&')
# print(clauses)
#bb.add("p -> q")
#bb.add("q -> r")
# print(bb.beliefBase)
# print(bb.CNF)

p & (p | s | ~r) & ~(D | ~(A | B | C))
p & ~D & (A | B | C) & (p | s | ~r)
p
~D
A | B | C
p | s | ~r


In [ ]:
cnf_conj_all.args

(p, ~D, A | B | C, p | s | ~r)

In [ ]:
x = Symbol('x')
y = Symbol('y')
A = Or(x,y)
B = Or(x,Not(y))
r = bb.presolve(A,B)

print(r)

[x, y] [x, ~y]
[x] [x]
x


In [ ]:
x = Symbol('x')
y = Symbol('y')
z = Symbol('z')
A = Or(x,y,z)
B = Or(x,Not(y),z)
r = bb.presolve(A,B)
print(r)

[x, y, z] [x, z, ~y]
[x, z] [x, z]
x | z


In [ ]:
x = Symbol('x')
y = Symbol('y')
z = Symbol('z')
A = Or(x,y,z)
B = Or(x,Not(y),Not(z))
r = bb.presolve(A,B)
print(r)

[x, y, z] [x, ~y, ~z]
[x, z] [x, ~z]
[x] [x]
x


In [ ]:
x = Symbol('x')
y = Symbol('y')
A = Or(x,y)
B = Or(Not(x),Not(y))
r = bb.presolve(A,B)
if len(r.args) == 0:
    print("len")
print(r)

[x, y] [~x, ~y]
[y] [~y]
[] []
len
False


In [ ]:
a = Symbol('a')
b = Not(a)

eq = a == b

if eq:
    print("eq")
else:
    print("not")


not
